In [4]:
import requests

ip = '62.68.146.67'
#ip = 'localhost'
#url = f"http://{ip}:8000/reset_database"

#response = requests.post(url)
#response.json()

In [258]:
import requests

url = f"http://{ip}:8000/add_from_sheet"

sheet_data = {
    "sheet_id": "1qEormopG5VJDg9BfG4-d5kRBTdMPksRcN2sXar9Xepo",
    "gid": "0"
}

response = requests.post(url, json=sheet_data)
response.json()

{'status': 'success',
 'message': 'Added 1659 new unique question-answer pairs',
 'new_pairs_count': 1659,
 'duplicates_skipped': 0,
 'total_rows_in_sheet': 1659,
 'used_question_column': 'question',
 'used_answer_column': 'content'}

In [5]:
import requests

url = f"http://{ip}:8000/view_top_n"

params = {
    "n": 5
}

response = requests.get(url, params=params)
response.json()

{'total_items': 5,
 'items': [{'id': '0',
   'question': 'Какие документы необходимо предоставить при приеме беженцев?',
   'answer': 'Вариант 1: 1) действующее удостоверение беженца: документ, удостоверяющий личность на территории РФ; 2) регистрация по месту пребывания; 3) СНИЛС (при наличии); 4) трудовая книжка и/или сведения о трудовой деятельности (при наличии трудового стажа). Вариант 2: 1) действующее свидетельство о предоставлении временного убежища: документ, удостоверяющий личность на территории РФ; 2) может быть предоставлен паспорт, но основанием для трудоустройства будет именно свидетельство; 3) регистрация по месту пребывания; 4) СНИЛС (при наличии); 5) трудовая книжка и/или сведения о трудовой деятельности (при наличии трудового стажа). Вариант 3: 1) паспорт гражданина Украины, Донецкой Народной Республики или Луганской Народной Республики; 2) документ, подтверждающий прохождение дактилоскопии и фотографирования. Данные должны совпадать с данными в паспорте; 3) регистраци

In [260]:
import requests

url = f"http://{ip}:8000/count_items"

response = requests.get(url)
print(response.json())

{'collection_name': 'default', 'item_count': 1659}


In [6]:
import requests

url = f"http://{ip}:8000/api/v1/get_answer/"

data = {
    "queries": ["как провести отпуск"],
    "n_results": 3
}

response = requests.post(url, json=data)
response.json()

{'results': [{'query': 'как провести отпуск',
   'results': [{'id': '113',
     'question': 'Как провести отпуск?',
     'answer': 'Оформление отпуска по заявлению возможно двумя способами: 1. в разделе "Отпуска" в левом меню Личного кабинета сотрудника (https://company-x5.ru/vacations/plan?vp_page=1 ); 2. в разделе "Заявки", группа "Отпуск", плитка "Отпуск по заявлению (ЛК)". (https://company-x5.ru/requests/tiles/my/)',
     'distance': 0.17390930652618408},
    {'id': '448',
     'question': 'как оформить отпуск',
     'answer': 'Создать заявку на отпуск можно в разделе "Сервис", подразделе "Отпуск". Требования к запрашиваемому отпуску вы сможете видеть левой части экрана',
     'distance': 0.2272828221321106},
    {'id': '447',
     'question': 'как создать заявку на отпуск?',
     'answer': 'Создать заявку на отпуск можно в разделе "Сервис", подразделе "Отпуск". Требования к запрашиваемому отпуску вы сможете видеть левой части экрана',
     'distance': 0.2599191665649414}]}]}

In [2]:
import requests
import json

url = "http://62.68.146.67:11434/api/chat/"

# Пример использования
prompt = "Привет"

payload = {
    "model": "gemma2:2b-instruct-q8_0",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "stream": False
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))
response.json()

{'model': 'gemma2:2b-instruct-q8_0',
 'created_at': '2024-09-13T07:04:39.62538625Z',
 'message': {'role': 'assistant', 'content': 'Привет! 😊  Чем могу помочь? \n'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 356570995,
 'load_duration': 44079352,
 'prompt_eval_count': 10,
 'prompt_eval_duration': 33177000,
 'eval_count': 11,
 'eval_duration': 277771000}

In [3]:
import requests
import json
from typing import List, Dict, Any
from collections import defaultdict

def rag_query(ip: str, queries: List[str], n_results: int) -> Dict[str, Any]:
    """
    Perform a RAG query to retrieve relevant information.
    """
    url = f"http://{ip}:8000/api/v1/get_answer/"
    data = {
        "queries": queries,
        "n_results": n_results
    }
    response = requests.post(url, json=data)
    result = response.json()
    #print("RAG Response:", json.dumps(result, indent=2, ensure_ascii=False))
    return result

def llm_query(prompt: str, model: str = "gemma2:9b") -> Dict[str, Any]:
    """
    Perform an LLM query using the given prompt.
    """
    url = f"http://{ip}:11434/api/chat/"
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "options": {
            "temperature": 0
        },
    }
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    return response.json()


def create_llm_prompt(user_question: str, rag_response: Dict[str, Any]) -> str:
    """
    Create a prompt for the LLM based on the user question and RAG results,
    with question grouping for identical answers.
    """
    prompt = f'Вопрос от пользователя: "{user_question}"\n\nРелевантные вопросы и ответы из базы знаний:\n'
    
    # Extract results from the RAG response
    rag_results = rag_response['results'][0]['results']
    
    # Group questions by their answers
    answer_to_questions = defaultdict(list)
    for result in rag_results:
        answer_to_questions[result['answer']].append(result['question'])
    
    # Create condensed Q&A pairs
    for i, (answer, questions) in enumerate(answer_to_questions.items(), 1):
        if len(questions) > 1:
            formatted_questions = ', '.join(f'"{q}"' for q in questions)
            prompt += f"#{i}. Для вопросов: {formatted_questions}\n   Ответ: {answer}\n\n"
        else:
            prompt += f"#{i}. \"{questions[0]}\"\n   Ответ: {answer}\n\n"
    
    prompt += '''
Задача: Основываясь на вспомогательных материалах корпоративной базы знаний компании x5 retail group, развёрнуто и чётко ответить на вопрос пользователя. 
Важно, что вспомогательные данные не всегда ранжированы по релевантности. 
Среди примеров может не быть верного ответа! 
Если ты считаешь, вопрос пользователя релевантен вопросу из базы знаний, дай соответствующий ответ. 
Тебе необходимо ответить на вопрос. Объяснять, почему ты выбрал именно это вариант из базы не нужно. 
Если ты считаешь, что в предоставленных данных не содержится ответа на вопрос, то нужно ответить, что тебе дали слишком мало конкретики. Попроси описать проблему подробнее. 
Если в вспомогательных данных есть несколько вопросов, которые релевантны данному, задай уточняющий вопрос, чтобы понять, какой из вопросов больше подходит.
При ответе на вопрос ничего лишнего не пиши. Только сам ответ или уточняющее предложение.
Ответ должен быть полным, как в базе знаний. Из базы знаний нужно выбрать лишь один ответ!
Отвечай только на русском языке.'''
    
    return prompt

def rag_llm_pipeline(ip: str, user_question: str, n_results: int = 5) -> Dict[str, Any]:
    """
    Perform the full RAG-LLM pipeline for a given user question.
    """
    # 1. Perform RAG query
    rag_response = rag_query(ip, [user_question], n_results)
    
    # 2. Create LLM prompt
    llm_prompt = create_llm_prompt(user_question, rag_response)
    
    # 3. Perform LLM query
    llm_response = llm_query(llm_prompt)
    
    return {
        "user_question": user_question,
        "rag_results": rag_response,
        "llm_prompt": llm_prompt,
        "llm_response": llm_response
    }

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma2")
llm.invoke("Why is the sky blue?")

In [345]:
answer = rag_llm_pipeline(ip, "процент занятости нужно изменить")

In [346]:
answer['llm_response']

{'model': 'gemma2:9b',
 'created_at': '2024-09-12T17:55:57.067455211Z',
 'message': {'role': 'assistant',
  'content': 'Для изменения процента занятости сотруднику создайте, пожалуйста, заявку на сотрудника по теме "Изменение режима, характера работы" \n'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 1733768778,
 'load_duration': 62059932,
 'prompt_eval_count': 398,
 'prompt_eval_duration': 634273000,
 'eval_count': 31,
 'eval_duration': 1034048000}

In [4]:
from langchain_community.llms import Ollama

# Указываем IP-адрес и порт сервера Ollama
ollama_model = Ollama(model="gemma2:9b", base_url='http://62.68.146.67:11434')

In [5]:
ollama_model.invoke("Why is the sky blue?")

"The sky appears blue due to a phenomenon called **Rayleigh scattering**.\n\nHere's how it works:\n\n* **Sunlight:** Sunlight is actually composed of all colors of the rainbow.\n* **Atmosphere:** Earth's atmosphere is made up of tiny particles, mainly nitrogen and oxygen molecules.\n* **Scattering:** When sunlight enters the atmosphere, it collides with these particles.  Blue light, having a shorter wavelength, is scattered more effectively than other colors by these particles.\n\n* **Our Perception:** This scattered blue light reaches our eyes from all directions, making the sky appear blue.\n\n**Why not violet?** \nViolet light has an even shorter wavelength than blue and scatters even more. However, our eyes are less sensitive to violet light, so we perceive the sky as blue.\n\n**Sunsets and sunrises:**\n\nDuring sunrise and sunset, the sunlight travels through a thicker layer of atmosphere. This causes more scattering of all colors, but especially the longer wavelengths like orange

In [32]:
from typing import List, Dict, Any
import requests
import json

def process_history(history: List[Dict[str, str]], ip: str, N: int = 2, M: int = 3) -> List[Dict[str, str]]:
    # Получаем последние N сообщений пользователя
    user_messages = [msg["content"] for msg in history if msg["role"] == "user"][-N:]
    
    # Получаем конкатенацию последних M сообщений (всех ролей)
    all_messages = " ".join([msg["content"] for msg in history[-M:]])
    
    # Формируем список запросов для RAG
    queries = user_messages + [all_messages]
    
    # Выполняем запрос к RAG
    rag_results = rag_query(ip, queries, n_results=5)
    
    # Обрабатываем результаты RAG
    processed_results = []
    seen_pairs = set()  # Для отслеживания уникальных пар вопрос-ответ

    for query_result in rag_results['results']:
        for result in query_result['results']:
            qa_pair = (result['question'], result['answer'])
            if qa_pair not in seen_pairs:
                processed_results.append({
                    'question': result['question'],
                    'answer': result['answer']
                })
                seen_pairs.add(qa_pair)
    
    return processed_results

def create_condensed_qa_prompt(user_question: str, qa_pairs: List[Dict[str, str]]) -> str:
    """
    Create a condensed prompt from a list of question-answer dictionaries,
    grouping questions with identical answers, and including the latest user question.
    """
    prompt = f'Вопрос от пользователя: "{user_question}"\n\nРелевантные вопросы и ответы из базы знаний:\n'
    
    # Group questions by their answers
    answer_to_questions = defaultdict(list)
    for pair in qa_pairs:
        answer_to_questions[pair['answer']].append(pair['question'])
    
    # Create condensed Q&A pairs
    for i, (answer, questions) in enumerate(answer_to_questions.items(), 1):
        if len(questions) > 1:
            formatted_questions = ', '.join(f'"{q}"' for q in questions)
            prompt += f"#{i}. Для вопросов: {formatted_questions}\n   Ответ: {answer}\n\n"
        else:
            prompt += f"#{i}. \"{questions[0]}\"\n   Ответ: {answer}\n\n"
    
    prompt += '''
Задача: Основываясь на вспомогательных материалах корпоративной базы знаний компании x5 retail group, развёрнуто и чётко ответить на вопрос пользователя. 
Важно, что вспомогательные данные не всегда ранжированы по релевантности. 
Среди примеров может не быть верного ответа! 
Если ты считаешь, вопрос пользователя релевантен вопросу из базы знаний, дай соответствующий ответ. 
Тебе необходимо ответить на вопрос. Объяснять, почему ты выбрал именно это вариант из базы не нужно. 
Если ты считаешь, что в предоставленных данных не содержится ответа на вопрос, то нужно ответить, что тебе дали слишком мало конкретики. Попроси описать проблему подробнее. 
Если в вспомогательных данных есть несколько вопросов, которые релевантны данному, задай уточняющий вопрос, чтобы понять, какой из вопросов больше подходит.
При ответе на вопрос ничего лишнего не пиши. Только сам ответ или уточняющее предложение.
Ответ должен быть полным, как в базе знаний. Из базы знаний нужно выбрать лишь один ответ!
Отвечай только на русском языке.'''
    
    return prompt


def remove_role_messages(history, role="assistant", n=2):
    """
    Удаляет все сообщения указанной роли, кроме последних N.
    
    :param history: список сообщений, содержащих "role" и "content"
    :param role: роль, сообщения которой нужно удалить (например, "assistant")
    :param n: количество последних сообщений этой роли, которые нужно оставить
    :return: список сообщений с удалёнными сообщениями указанной роли
    """
    # Собираем все индексы сообщений указанной роли
    role_indices = [i for i, message in enumerate(
        history) if message['role'] == role]

    # Оставляем последние N индексов для сохранения
    indices_to_keep = role_indices[-n:]

    # Формируем новый список, удаляя все сообщения роли, кроме последних N
    new_history = [message for i, message in enumerate(
        history) if message['role'] != role or i in indices_to_keep]

    return new_history

#@app.post("/api/v1/get_answer/")
def query(chat_history):
    # Нахождение релевантной информации
    result = process_history(chat_history, ip)
    # Создание сообщения пользователя
    res = create_condensed_qa_prompt(chat_history[-1]['content'], result)
    # Формирование истории
    result_history = remove_role_messages(chat_history, 'assistant', 2)
    result_history = result_history[:-1]
    result_history.append({'role': 'user', 'content': res})
    return result_history

In [33]:

# Пример использования
history = [
    {'role': 'user', 'content': 'Как мне взять отпуск?'},
    {'role': 'assistant', 'content': 'Какой тип отпуска вас интересует?'},
    {'role': 'user', 'content': 'Ежегодный оплачиваемый отпуск'},
    {'role': 'assistant', 'content': 'Сколько дней отпуска вы хотите взять?'},
    {'role': 'user', 'content': '14 дней'},
    {'role': 'assistant', 'content': 'Для оформления ежегодного оплачиваемого отпуска на 14 дней, вам нужно подать заявление за две недели до предполагаемой даты начала. Вы уже выбрали даты?'},
    {'role': 'user', 'content': 'Да, с 1 по 14 августа.'},
    {'role': 'assistant', 'content': 'Отлично. Убедитесь, что ваш начальник одобрил даты. Вам нужна помощь с заполнением заявления на отпуск?'},
    {'role': 'user', 'content': 'Да, подскажите, как его правильно заполнить.'},
    {'role': 'assistant', 'content': 'В заявлении укажите ваши ФИО, должность, тип отпуска (в вашем случае - ежегодный оплачиваемый), а также даты начала и окончания отпуска. Не забудьте подписать и указать текущую дату. Затем подайте заявление в отдел кадров.'},
    {'role': 'user', 'content': 'Спасибо, понял. А что делать, если не хватит дней отпуска?'},
]

query(history)

[{'role': 'user', 'content': 'Как мне взять отпуск?'},
 {'role': 'user', 'content': 'Ежегодный оплачиваемый отпуск'},
 {'role': 'user', 'content': '14 дней'},
 {'role': 'user', 'content': 'Да, с 1 по 14 августа.'},
 {'role': 'assistant',
  'content': 'Отлично. Убедитесь, что ваш начальник одобрил даты. Вам нужна помощь с заполнением заявления на отпуск?'},
 {'role': 'user', 'content': 'Да, подскажите, как его правильно заполнить.'},
 {'role': 'assistant',
  'content': 'В заявлении укажите ваши ФИО, должность, тип отпуска (в вашем случае - ежегодный оплачиваемый), а также даты начала и окончания отпуска. Не забудьте подписать и указать текущую дату. Затем подайте заявление в отдел кадров.'},
 {'role': 'user',
  'content': 'Вопрос от пользователя: "Спасибо, понял. А что делать, если не хватит дней отпуска?"\n\nРелевантные вопросы и ответы из базы знаний:\n#1. Для вопросов: "задача в лк", "сделать заявку", "Прошу закрыть заявку. Уже не актуальна", "сделать заявку на о не корректноц информ

In [46]:
import requests

chat_hist = {
  "history": [
    {
      "role": "user",
      "content": "Добрый день! Хотел бы уточнить, как можно оформить отпуск в следующем месяце?"
    },
    {
      "role": "assistant",
      "content": "Здравствуйте! Для оформления отпуска вам нужно заполнить заявление на отпуск и согласовать даты с руководителем. После этого заявление отправляется в отдел кадров для оформления приказа."
    },
    {
      "role": "user",
      "content": "Спасибо! Есть ли ограничения по продолжительности отпуска?"
    },
    {
      "role": "assistant",
      "content": "Обычно отпуск предоставляется на срок до 28 календарных дней в год. Однако, вы можете разбить его на несколько частей, при этом одна из частей не может быть меньше 14 дней."
    },
    {
      "role": "user",
      "content": "Понял. А можно ли перенести часть отпуска на следующий год?"
    }
  ]
}

response = requests.post("http://localhost:8000/api/v1/get_answer/", json=chat_hist)
response.json()

{'model': 'gemma2:9b',
 'created_at': '2024-09-12T21:11:15.442003096Z',
 'message': {'role': 'assistant',
  'content': 'Согласно информации из базы знаний, перенос отпуска возможен не позднее чем за 6 рабочих дней до его начала. Для этого вам нужно перейти в раздел «Отпуска» https://company-x5.ru/vacations/plan?vp_page=1 , а затем нажать кнопку «Выберите действие», далее «Перенос планового отпуска».\n\n\n'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 4401551520,
 'load_duration': 237287701,
 'prompt_eval_count': 1147,
 'prompt_eval_duration': 1370415000,
 'eval_count': 78,
 'eval_duration': 2683977000}